In [1]:
!pip install  crewai crewai_tools langchain langchain_community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 15.9 MB/s et

In [37]:
# Create an LLM with a temperature of 0 to ensure deterministic outputs
llm = LLM(model="gpt-4o-mini", temperature=0, api_key = "**************************")

In [38]:
import os
os.environ["OPENAI_API_KEY"] ="****************"

In [ ]:
########### ANother Way (RAG) ##########

In [43]:
from crewai_tools import PDFSearchTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

In [44]:
# Initialize the tool allowing for any PDF content search if the path is provided during execution
tool = PDFSearchTool()

In [45]:
# Initialize the tool with a specific PDF path for exclusive search within that document
tool = PDFSearchTool(pdf='/content/ORMC UHC Commercial PA eff 1.1.18.pdf')

Inserting batches in chromadb: 100%|██████████| 2/2 [00:04<00:00,  2.08s/it]


In [49]:
# Create an agent with the knowledge store
agent = Agent(
    role="Documnet researcher",
    goal="You read carefully everything in document and answer the question with 100% accuracy",
    backstory="""You are a master at understanding content and respond logically.""",
    tools=[tool],
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [50]:
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

In [51]:
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
)

In [53]:
result = crew.kickoff(inputs={"question": "What is the Base rate for all MS-DRG's?"})

# Agent: Documnet researcher
## Task: Answer the following questions about the user: What is the Base rate for all MS-DRG's?
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedPDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': "Base rat...S-DRG's", 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query the /content/ORMC UHC Commercial PA eff 1.1.18.pdf PDF's content.



# Agent: Documnet researcher
## Thought: I need to find the Base rate for all MS-DRG's in the provided PDF document.
## Using tool: Search a PDF's content


In [54]:
result = crew.kickoff(inputs={"question": "Can you provide Services Not Payable Under This Appendix But May be Payable Under Another Appendix To The Agreement Or Under Another Agreement. in tabular format?"})

# Agent: Documnet researcher
## Task: Answer the following questions about the user: Can you provide Services Not Payable Under This Appendix But May be Payable Under Another Appendix To The Agreement Or Under Another Agreement. in tabular format?


# Agent: Documnet researcher
## Thought: I need to find the specific section in the document that lists the services not payable under the specified appendix but may be payable under another appendix or agreement.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"Services Not Payable Under This Appendix But May be Payable Under Another Appendix To The Agreement Or Under Another Agreement\"}"
## Tool Output: 
Relevant Content:
appendix to the Agreement applies to these services rendered to a Customer the services below may be payable under that agreement or appendix. Table 6: Services Not Payable Under This Appendix But May be Payable Under Another Appendix To The Agreement Or Under Another Agreement.

Massage ~ If Eligibl

In [ ]:
result = crew.kickoff(inputs={"question": "What is Revenue codes for Outpatient Procedures"})

# Agent: Documnet researcher
## Task: Answer the following questions about the user: What is Revenue codes for Outpatient Procedures
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedPDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': 'Revenue ...cedures', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query the /content/ORMC UHC Commercial PA eff 1.1.18.pdf PDF's content.



# Agent: Documnet researcher
## Thought: I need to find the specific revenue codes for outpatient procedures in the provided PDF document.
## Using

In [57]:
result = crew.kickoff(inputs={"question": "What is Contract Rate for Chemotherapy Administration"})

# Agent: Documnet researcher
## Task: Answer the following questions about the user: What is Contract Rate for Chemotherapy Administration
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for FixedPDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': 'Contract...tration', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query the /content/ORMC UHC Commercial PA eff 1.1.18.pdf PDF's content.



# Agent: Documnet researcher
## Thought: I need to find the specific information regarding the Contract Rate for Chemotherapy Administration 